source: https://github.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter1_Introduction/Ch1_Introduction_Pyro.ipynb


In [ ]:
pip install pyro-ppl

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

import torch
from torch.distributions import constraints
from torch import tensor

import pyro
import pyro.distributions as dist
from pyro.infer import SVI,Trace_ELBO
from pyro.infer import MCMC, NUTS, HMC
from pyro.infer.autoguide  import AutoMultivariateNormal, AutoNormal, init_to_mean
from pyro.optim import ClippedAdam

assert pyro.__version__.startswith('1.8')
pyro.set_rng_seed(1)
torch.manual_seed(1)

# Set matplotlib settings
%matplotlib inline
plt.style.use('default')
plt.rcParams['figure.figsize'] = [15, 5]

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/master/Chapter1_Introduction/data/txtdata.csv", header=None)
df.columns = ['messages_received']
df.loc[:,'day'] = df.index +1
df.head()

In [ ]:
df.describe().T

In [ ]:
sns.displot(x = df['messages_received']);

You are given a series of daily text-message counts from a user of your system. The data, plotted over time, appears in the chart below. How would you model this data

In [ ]:
sns.barplot(y='messages_received', x='day', data=df, palette='colorblind')
plt.xticks(fontsize=8);

In [ ]:
X = df.copy()

In [ ]:
X['messages_received'].mean(), X['messages_received'].std()

In [ ]:
# Let's look at an actual poisson distribution fit to the mean of the data
samples = dist.Poisson(X['messages_received'].mean()).sample(tensor([74])) # create sample distribution where Poisson mean = data mean
sample_df = pd.DataFrame({'sample_texts_received':samples.numpy(), 'day':X['day'].copy()})

In [ ]:
sns.displot(x = sample_df['sample_texts_received']);

### So, comparing the actual data with the above two poisson sampling distributions above, it appears our data is overdispersed. Our data is heavily skewed to the right, ie., it has many more texts received far away from the mean, and much greater variability spikes much bigger than a poisson distribution would suggest.

### Despite the fact that we know the poisson is not the 'correct' distribution to handle the outliers here, let's continue with the analysis now, and we can try other distributions in future work

In [ ]:
data = tensor(X['messages_received'].values, dtype=torch.float)

In [ ]:
data.size(0)

In [ ]:
# this is a trick to marginalize over all 74 days of the discrete variable
def model(data):
    alpha = 1.0 / data.mean()
    lambda_1 = pyro.sample("lambda_1", dist.Exponential(alpha))
    lambda_2 = pyro.sample("lambda_2", dist.Exponential(alpha))
    
    tau = pyro.sample("tau", dist.Uniform(0, 1))
    lambda1_size = (tau * data.size(0) + 1).long()
    lambda2_size = data.size(0) - lambda1_size
    lambda_ = torch.cat([lambda_1.expand((lambda1_size,)),
                         lambda_2.expand((lambda2_size,))])

    with pyro.plate("data", data.size(0)):
        pyro.sample("obs", dist.Poisson(lambda_), obs=data)

In [ ]:
from pyro.infer.autoguide import AutoMultivariateNormal

guide = AutoMultivariateNormal(model, init_loc_fn=init_to_mean)

In [ ]:
def train(model, guide, lr=0.01, n_steps=4000):
    pyro.set_rng_seed(1)
    pyro.clear_param_store()
    
    gamma = 0.01  # final learning rate will be gamma * initial_lr
    lrd = gamma ** (1 / n_steps)
    adam = pyro.optim.ClippedAdam({'lr': lr, 'lrd': lrd})

    svi = SVI(model, guide, adam, loss=Trace_ELBO())

    for i in range(n_steps):
        elbo = svi.step(data)
        if i % 500 == 0:
          print(f"Elbo loss: {elbo}")
    print(f"Final Elbo loss: {elbo}")

In [ ]:
%%time
train(model, guide)

In [ ]:
from pyro.infer import Predictive

num_samples = 1000
predictive = Predictive(model, guide=guide, num_samples=num_samples)

svi_samples = {k: v.reshape((num_samples,-1)).detach().cpu().numpy()
               for k, v in predictive(data).items()
               if k != "obs"}

In [ ]:
svi_samples.keys()

In [ ]:
guide.quantiles([0.05,0.50,0.95])

Can see that SVI does not work in this instance, lambda_1 and lambda_2 overlap substantially.

In [ ]:
## Let's do MCMC instead

In [ ]:
kernel = NUTS(model, jit_compile=True, ignore_jit_warnings=True, max_tree_depth=3)
posterior = MCMC(kernel, num_samples=5000, warmup_steps=500)
posterior.run(data);

In [ ]:
hmc_samples = {k: v.detach().cpu().numpy() for k, v in posterior.get_samples().items()}
lambda_1_samples = hmc_samples['lambda_1']
lambda_2_samples = hmc_samples['lambda_2']
tau_samples = (hmc_samples['tau'] * data.size(0) + 1).astype(int)

In [ ]:
data.size(0)

In [ ]:
# plt.figsize(12.5, 10)
#histogram of the samples:

ax = plt.subplot(311)
ax.set_autoscaley_on(False)

plt.hist(lambda_1_samples, histtype='stepfilled', bins=30, alpha=0.85,
         label="posterior of $\lambda_1$", color="#A60628", density=True)
plt.legend(loc="upper left")
plt.title(r"""Posterior distributions of the variables
    $\lambda_1,\;\lambda_2,\;\tau$""")
plt.xlim([15, 30])
plt.xlabel("$\lambda_1$ value")

ax = plt.subplot(312)
ax.set_autoscaley_on(False)
plt.hist(lambda_2_samples, histtype='stepfilled', bins=30, alpha=0.85,
         label="posterior of $\lambda_2$", color="#7A68A6", density=True)
plt.legend(loc="upper left")
plt.xlim([15, 30])
plt.xlabel("$\lambda_2$ value")

plt.subplot(313)
w = 1.0 / tau_samples.shape[0] * np.ones_like(tau_samples)
plt.hist(tau_samples, bins=data.size(0), alpha=1,
         label=r"posterior of $\tau$",
         color="#467821", weights=w, rwidth=2.)
plt.xticks(np.arange(data.size(0)))

plt.legend(loc="upper left")
plt.ylim([0, .75])
# plt.xlim([35, len(data.size(0))-20])
plt.xlabel(r"$\tau$ (in days)")
plt.ylabel("probability");

### Interpretation
Recall that Bayesian methodology returns a distribution. Hence we now have distributions to describe the unknown s and . What have we gained? Immediately, we can see the uncertainty in our estimates: the wider the distribution, the less certain our posterior belief should be. We can also see what the plausible values for the parameters are: 
 is around 18 and 
 is around 23. The posterior distributions of the two s are clearly distinct, indicating that it is indeed likely that there was a change in the user's text-message behaviour.

What other observations can you make? If you look at the original data again, do these results seem reasonable?

Notice also that the posterior distributions for the s do not look like exponential distributions, even though our priors for these variables were exponential. In fact, the posterior distributions are not really of any form that we recognize from the original model. But that's OK! This is one of the benefits of taking a computational point of view. If we had instead done this analysis using mathematical approaches, we would have been stuck with an analytically intractable (and messy) distribution. Our use of a computational approach makes us indifferent to mathematical tractability.

Our analysis also returned a distribution for . Its posterior distribution looks a little different from the other two because it is a discrete random variable, so it doesn't assign probabilities to intervals. We can see that near day 45, there was a 50% chance that the user's behaviour changed. Had no change occurred, or had the change been gradual over time, the posterior distribution of  would have been more spread out, reflecting that many days were plausible candidates for . By contrast, in the actual results we see that only three or four days make any sense as potential transition points.

In [ ]:
# The switchpoint analysis done above would be considerably more difficult to do without Bayesian calculations